In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [ ]:
%%capture
!pip install openai
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install -U sentence-transformers
!pip install retry
# !pip install git+https://github.com/huggingface/transformers
# !pip install sentencepiece

In [ ]:
from evaluate import load
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

rouge = load('rouge')

predictions = ['hello']
references = ['hello world']


rouge_results = rouge.compute(predictions=predictions, references=references)
print( rouge_results['rougeL'], rouge_results )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.6666666666666666 {'rouge1': 0.6666666666666666, 'rouge2': 0.0, 'rougeL': 0.6666666666666666, 'rougeLsum': 0.6666666666666666}


In [ ]:
import os
import openai
from retry import retry

openai.api_key = ""

In [ ]:
# MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4"

@retry(
    (
        openai.APIError,
        openai.RateLimitError,
        openai.Timeout,
        # openai.ServiceUnavailableError,
        openai.APIConnectionError,
    ),
    tries=50,
    delay=2,
    backoff=2,
    max_delay=20,
)
def evaluate(each_input, stop=None):
    response = openai.chat.completions.create(
        model= MODEL,
        messages=[{"role": "user", "content": each_input}],
        max_tokens=1024,
        stop=stop,
    )
    return (response.choices[0].message.content).lower()

few-shot data

In [ ]:
import csv
import random
VAL_DATA = []

dir = '/gdrive/MyDrive/AI4EDU/val_OneSecPerLine.csv'


# Open file
with open(dir) as file_obj:
    heading = next(file_obj)

    reader_obj = csv.reader(file_obj)

    for row in reader_obj:
      eachQA = [ [row[3], '(' + row[3] + ', ' + row[4]  + ', ' + row[5] + ')', row[6], row[7]] ]
      if row[11] != '' and row[12] != '':
        eachQA.append( [row[8], '(' + row[8] + ', ' + row[9]  + ', ' + row[10] + ')', row[11], row[12]] )
      if row[16] != '' and row[17] != '':
        eachQA.append( [row[13], '(' + row[13] + ', ' + row[14]  + ', ' + row[15] + ')', row[16], row[17]] )
      VAL_DATA.append([row[2], eachQA])

print(len(VAL_DATA))
print(VAL_DATA[0])


380
['There was once a poor woman that had three daughters, and one day the\neldest said, "Mother, bake my cake and kill my cock till I go seek my\nfortune." So she did, and when all was ready, says her mother to her,\n"Which will you have--half of these with my blessing, or the whole with\nmy curse?" "Curse or no curse," says she, "the whole is little enough."\nSo away she set, and if the mother didn\'t give her her curse, she didn\'t\ngive her her blessing.\n\nShe walked, and she walked, till she was tired and hungry, and then she\nsat down to take her dinner. While she was eating it a poor woman came\nup, and asked for a bit. "The dickens a bit you\'ll get from me," says\nshe; "it\'s all too little for myself." And the poor woman walked away\nvery sorrowful. At nightfall she got lodging at a farmer\'s, and the\nwoman of the house told her that she\'d give her a spadeful of gold and a\nshovelful of silver if she\'d only sit up and watch her son\'s corpse that\nwas waking in the next 

test data

In [ ]:
import csv
TEST_DATA = []

dir = '/gdrive/MyDrive/AI4EDU/test_OneSecPerLine.csv'


# Open file
with open(dir) as file_obj:
    heading = next(file_obj)

    reader_obj = csv.reader(file_obj)


    for row in reader_obj:
      eachQA = [ [row[3], '(' + row[3] + ', ' + row[4]  + ', ' + row[5] + ')', row[6], row[7]] ]
      if row[11] != '' and row[12] != '':
        eachQA.append( [ row[8], '(' + row[8] + ', ' + row[9]  + ', ' + row[10] + ')', row[11], row[12]] )
      if row[16] != '' and row[17] != '':
        eachQA.append( [row[13], '(' + row[13] + ', ' + row[14]  + ', ' + row[15] + ')', row[16], row[17]] )
      TEST_DATA.append([row[2], eachQA])

print(len(TEST_DATA))
print(TEST_DATA[0])



365
['At the time when the Tang dynasty reigned over the Middle Kingdom, there were master swordsmen of various kinds. Those who came first were the saints of the sword. They were able to take different shapes at will, and their swords were like strokes of lightning. Before their opponents knew they had been struck their heads had already fallen. Yet these master swordsmen were men of lofty mind, and did not lightly mingle in the quarrels of the world. The second kind of master swordsmen were the sword heroes. It was their custom to slay the unjust, and to come to the aid of the oppressed. They wore a hidden dagger at their side and carried a leather bag at their belt. By magic means they were able to turn human heads into flowing water. They could fly over roofs and walk up and down walls, and they came and went and left no trace. The swordsmen of the lowest sort were the mere bought slayers. They hired themselves out to those who wished to do away with their enemies. And death was an

### GPT-3.5

##### instruction 1

story content --> QA

In [ ]:
INSTRUCTION_1 = f"""\
I need you to help generate a question and answer pair for young children aged three to six. I will provide you with a short section of a story delimited by triple quotes.
Please generate a question and answer pair that is derived from the story narrative, but associated with external commonsense knowledge.
Return the selected question-answer pair in the following format:

question: ...
answer: ...

"""


import random

def prompting(each_section):
  res = INSTRUCTION_1 + '<text>:\n\"\"\"' + each_section + '\"\"\"\n\n<response>:\n'

  return res

# ONE QA PER SECTION
def few_shot_prompting(each_section):

  res = ''
  cnt = 0

  random.shuffle(VAL_DATA)
  # while cnt < 3:
  for item in VAL_DATA:
    section = item[0]
    QA_list = item[1]
    # print(QA_list)
    if cnt == 5: break
    res += f"""Q: Now, generate a question and answer pair containing common-sense, fact-based knowledge associated with the following story for young children aged three to six.
    <story>:\n\"\"\"{section}\"\"\"\n"""
    # for i in QA_list:
    # ===== generating relation ===== #
    # res += f"""commonsense: {QA_list[0][0]}\nquestion: {QA_list[0][1]} \nanswer: {QA_list[0][2]}\n\n"""
    res += f"""
    A: Let's think step by step.
    First, a key word can be identified from the story text, which meets the following criteria: it is relatively complex, it is considered tier 1 or tier 2 vocabulary, and it is a concrete noun, verb, or adjective.
    The identified key word is: {QA_list[0][0]}
    Then, based on the identified key word, one piece of common-sense, fact-based knowledge can be generated in the form of a triple, such as A relation B, where A and B are two concepts and the selected word can be either A or B.
    The triple should use one of the following relations for the common-sense knowledge:
      causes
      desires
      has context of
      has property
      has subevent
      is a
      is at location of
      is capable of
      is created by
      is made of
      is part of
      is the antonym of
      is used for
    The generated common-sense knowledge is :{QA_list[0][1]}
    Finally, a question and answer pair can be generated based on the generated common-sense, fact-based knowledge. Either the question or the answer should contain that identified word. Each question should have one single correct answer that would be the same regardless of the children’s experiences.
    The generated question-answer pair is:
    question: {QA_list[0][2]}
    answer: {QA_list[0][3]}\n\n"""
    cnt += 1
    res += '\n\n'

  res += f"""Q: Now, generate a question and answer pair containing common-sense, fact-based knowledge associated with the following story for young children aged three to six:
    <story>:\n\"\"\"{each_section}\"\"\"\n"""
  return res

In [ ]:
import re
def extract_triple(string):
    pattern = r'\(([^,]+),([^,]+),([^)]+)\)'
    matches = re.findall(pattern, string)
    if matches:
        return matches[0]
    else:
        return None

In [ ]:
import time
id = 0
output = []
og_output = []
for each_data in TEST_DATA:
  print(id)
  id += 1
  # if id < 363:
  #   continue
  each_section = each_data[0]
  each_QA = each_section[1]
  # each_input = prompting( each_section )
  each_input = few_shot_prompting( each_section)
  # print('--------')
  # print(each_input)
  # print('--------')
  # break


  each_output = evaluate( each_input ).lower().split('\n')
  print(each_output)
  # if id == 10:
  #   break
  og_output.append(each_output)

  each_output_QA = []

  cs = ''
  q = ''
  a = ''
  # each_output_QA.append( [each_output[0].replace('question: ', '').strip(), each_output[1].replace('answer: ', '').strip()] )
  for i in each_output:
    if 'the generated common-sense knowledge is' in i:
      if cs == '':
        cs = '(' + ', '.join(extract_triple(i)) + ')'
    elif 'question: ' in i:
      if q == '':
        q = i.replace('question: ', '').strip()
    elif 'answer: ' in i:
      if a == '':
        a = i.replace('answer: ', '').strip()
  each_output_QA.append([cs, q, a])
  cs = ''
  q = ''
  a = ''
  '''
  q = ''
  a = ''
  # each_output_QA.append( [each_output[0].replace('question: ', '').strip(), each_output[1].replace('answer: ', '').strip()] )
  for i in each_output:
    if 'question: ' in i:
      if q == '':
        q = i.replace('question: ', '').strip()
    elif 'answer: ' in i:
      if a == '':
        a = i.replace('answer: ', '').strip()
  each_output_QA.append([q, a])
  q = ''
  a = ''
  '''
  print(each_output_QA)
  # if len(each_output) == 3:
  #   each_output_QA.append( [each_output[0].replace('commonsense relation: ', '').strip(), each_output[1].replace('question: ', '').strip(), each_output[2].replace('answer: ', '').strip()] )
  # elif len(each_output) == 2:
  #   each_output_QA.append( [each_output[0].replace('question: ', '').strip(), each_output[1].replace('answer: ', '').strip()] )
  # each_output_QA.append( [each_output[3].replace('question: ', '').strip(), each_output[4].replace('answer: ', '').strip()] )

  # each_output_QA.append( [each_output[0].split('answer: ')[0].replace('question: ', '').strip(), each_output[0].split('answer: ')[1].strip()] )
  # each_output_QA.append( [each_output[1].split('answer: ')[0].replace('question: ', '').strip(), each_output[1].split('answer: ')[1].strip()] )

  output.append( each_output_QA )

# assert len(output) == len(TEST_DATA)



0
["a: let's think step by step.", '    first, a key word can be identified from the story text, which meets the following criteria: it is relatively complex, it is considered tier 1 or tier 2 vocabulary, and it is a concrete noun, verb, or adjective.', '    the identified key word is: sword', '    then, based on the identified key word, one piece of common-sense, fact-based knowledge can be generated in the form of a triple, such as a relation b, where a and b are two concepts and the selected word can be either a or b.', '    the triple should use one of the following relations for the common-sense knowledge:', '      causes', '      desires', '      has context of', '      has property', '      has subevent', '      is a', '      is at location of', '      is capable of', '      is created by', '      is made of', '      is part of', '      is the antonym of', '      is used for', '    the generated common-sense knowledge is :(sword, is used for, defense)', '    finally, a question 

In [ ]:
output_dup = output

In [ ]:
output = output_dup + output

In [ ]:
print(len(output))

365


In [ ]:
with open('/gdrive/MyDrive/AI4EDU/GPT-4_CoT_story2qa_5fewshot_end2end_test3.txt', 'w') as file:
  for i in output:
    eachline = ""
    eachline += i[0][0] + ' | ' + i[0][1] + ' ' + i[0][2]
    file.write(eachline+'\n')


In [ ]:
print(out_put_data[99])

["what color were the princess's cheeks?", 'carnation']


In [ ]:
output = []
with open('/content/FLANT5XXL_1shot_QAGonly(1).txt', 'r') as file:
  for line in file.readlines():
      line = line.strip('\n')
      output.append([line])

In [ ]:
print(output[0])

What was the lowest sort of master swordsmen? hired slayers.


QA evaluation

In [ ]:
all_rouge =  all_sentsim = 0


for gt, gen_qa in zip(TEST_DATA, output):
# for gt, gen_qa in zip(TEST_DATA, out_put_data):
  gt_qa = gt[1]
  # gen_qa = [gen_qa]
  # print(gt_qa)
  # print(gen_qa)
  # break
  each_bleu = each_rouge = each_meteor = each_bertscore = each_sentsim = 0

  for gen_i in gen_qa:
    each_gen_qa = ' '.join(gen_i[1:])
    # each_gen_qa = ' '.join(gen_i)
    # each_gen_qa = gen_i
    print(each_gen_qa)
    embeddings1 = model.encode(each_gen_qa, convert_to_tensor=True)

    for gt_i in (gt_qa):
      each_gt_qa = ' '.join(gt_i[2:])
      print(each_gt_qa)
      # new_bleu = bleu.compute(predictions=[each_gen_qa], references=[each_gt_qa])['bleu']
      # if new_bleu > each_bleu:
      #   each_bleu = new_bleu

      new_rouge = rouge.compute(predictions=[each_gen_qa], references=[each_gt_qa])['rougeL']
      if new_rouge > each_rouge:
        each_rouge = new_rouge

      # new_bertscore = bertscore.compute(predictions=[each_gen_qa], references=[each_gt_qa], lang="en")['f1'][0]
      # if new_bertscore > each_bertscore:
      #   each_bertscore = new_bertscore

      embeddings2 = model.encode(each_gt_qa, convert_to_tensor=True)
      cosine_scores = util.cos_sim(embeddings1, embeddings2)[0][0].item()

      if cosine_scores > each_sentsim:
        each_sentsim = cosine_scores

  # all_bleu += each_bleu
  all_rouge += each_rouge
  # all_bertscore += each_bertscore
  all_sentsim +=each_sentsim


# print("bleu: ", all_bleu / len(output))
print("rouge: ", all_rouge / len(output))
# print("bertscore: ", all_bertscore / len(output))
print("sentsim: ", all_sentsim / len(output))

what is a sword used for? defense.
What is a bag used for? A bag is used for carrying things.
What is a short sword called? A dagger.
what is the opposite of forget? remember.
What is a peach? A peach is a fruit.
What do you read a story in? In a book.
what is a servant capable of? a servant is capable of serving people.
What is a rooster? A rooster is a male chicken.
What causes someone to cry? They are sad.
what is the opposite of being happy? being sad.
What shape is round? A circle is round.
What do you see when you look in the mirror? I see myself.
what do people sometimes use dogs for at home? for protection.
What do tigers like to eat? Tigers like to eat meat.
What do you do when a room is dark? Turn on the light.
what can magic be used for? doing impossible things.
What is a lamp used for? A lamp is used to create light.
What animal barks? A dog.
what is a curtain used for? privacy.
What is a curtain used for? A curtain is used for covering a window.
Where can you find a curtai

commonsense knowledge evaluation

In [ ]:
all_rouge = all_sentsim = 0


for gt, gen_cs in zip(TEST_DATA, output):
# for gt, gen_qa in zip(TEST_DATA, out_put_data):
  gt_cs = gt[1]
  # print(gt_qa)
  # print(gen_qa)
  # break
  # gen_cs = [gen_cs]
  # print(gt_cs)
  # print(gen_cs)
  # break

  each_rouge = each_sentsim = 0

  for gen_i in gen_cs:
    each_gen_cs = gen_i[0]
    print(each_gen_cs)
    embeddings1 = model.encode(each_gen_cs, convert_to_tensor=True)

    for gt_i in (gt_cs):
      each_gt_cs = gt_i[1]
      print(each_gt_cs)
      # new_bleu = bleu.compute(predictions=[each_gen_qa], references=[each_gt_qa])['bleu']
      # if new_bleu > each_bleu:
      #   each_bleu = new_bleu

      new_rouge = rouge.compute(predictions=[each_gen_cs], references=[each_gt_cs])['rougeL']
      if new_rouge > each_rouge:
        each_rouge = new_rouge

      # new_bertscore = bertscore.compute(predictions=[each_gen_qa], references=[each_gt_qa], lang="en")['f1'][0]
      # if new_bertscore > each_bertscore:
      #   each_bertscore = new_bertscore

      embeddings2 = model.encode(each_gt_cs, convert_to_tensor=True)
      cosine_scores = util.cos_sim(embeddings1, embeddings2)[0][0].item()

      if cosine_scores > each_sentsim:
        each_sentsim = cosine_scores

  # all_bleu += each_bleu
  all_rouge += each_rouge
  # all_bertscore += each_bertscore
  all_sentsim +=each_sentsim


# print("bleu: ", all_bleu / len(output))
print("rouge: ", all_rouge / len(output))
# print("bertscore: ", all_bertscore / len(output))
print("sentsim: ", all_sentsim / len(output))

(sword,  is used for,  defense)
(bag, is used for, carrying_things)
(dagger, is a, short_sword)
(forget,  is the antonym of,  remember)
(peach, is a, fruit)
(story, is part of, book)
(servant,  is capable of,  serving_people)
(rooster, is a, male_chicken)
(sad, has subevent, crying)
(happy,  is the antonym of,  sad)
(round, is a, shape)
(mirror, is used for, see_yourself)
(dog,  is used for,  protection)
(tiger, is a, carnivore)
(dark, causes, turn_on_light)
(magic,  is used for,  doing_impossible_things)
(lamp, is used for, illuminating_area)
(dog, is capable of, bark)
(curtain,  is used for,  privacy)
(curtain, is used for, cover_window)
(curtain, is at location of, window)
(slave,  is a,  servant)
(jade, has property, green)
(call, is used for, contact)
(prince,  is a,  royal)
(discover, has subevent, learn)
(sword, has property, sharp)
(blame,  has context of,  mistakes)
(wagon, is used for, carry_children)
(danger, is the antonym of, safety)
(dagger,  is used for,  defense)
(hawk,